# VIPCCA scRNA-seq + scATAC-seq tutorial
This tutorial shows loading, preprocessing, VIPCCA integration and visualization of scRNA-seq and scATAC-seq dataset.

we obtained a [scRNA-seq data](http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.h5) consisting of gene expression measurements on 33,538 genes in 11,769 cells and a [scATAC-seq data](http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_10k/atac_v1_pbmc_10k_filtered_peak_bc_matrix.h5) consisting of 89,796 open chromatin peaks on 8,728 nuclei, both were produced by 10X Genomics Chromium system and were on PBMCs.

### Preprocessing with R
For the scRNA-seq data, we obtained 13 cell types using a [standard workflow](https://www.dropbox.com/s/3f3p5nxrn5b3y4y/pbmc_10k_v3.rds?dl=1) in Seurat. The 13 cell types include 460 B cell progenitor, 2,992 CD14+ Monocytes, 328 CD16+ Monocytes, 1,596 CD4 Memory, 1,047 CD4 Naïve, 383 CD8 effector, 337 CD8 Naïve, 74 Dendritic cell, 592 Double negative T cell, 544 NK cell, 68 pDC, 52 Plateletes, and 599 pre-B cell.

For the scATAC-seq data, we filtered out cells that have with fewer than 5,000 total peak counts to focus on a final set of 7,866 cells for analysis. See the Signac website for [up-to-date vignettes](https://satijalab.org/signac/articles/pbmc_vignette.html) and documentation for analysing scATAC-seq data.

After filter, we converting Seurat Object to AnnData via h5Seurat using R packages. In this case, the atac.h5ad file will be generated in the corresponding path.
```R
library(Seurat)
library(SeuratData)
library(SeuratDisk)

SaveH5Seurat(atac, filename = "atac.h5Seurat")
Convert("atac.h5Seurat", dest = "h5ad")
```

### Importing VIPCCA

In [1]:
import vipcca.vipcca as vip
import vipcca.preprocessing as pp
import matplotlib
import scanpy as sc
matplotlib.use('TkAgg')

# Command for Jupyter notebooks only
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

Using TensorFlow backend.


### Loading data in python

In [3]:
file_rna = "/Users/zhongyuanke/data/vipcca/rna.h5ad"
file_atac = "/Users/zhongyuanke/data/vipcca/atac.h5ad"